In [61]:
import torch ##it does the linear algebra
device  ='cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

cpu


In [20]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [47]:
##we will use a tokenizer it consist of encoder and decorder...encorder converts all the chars of the array into integer 
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])
"""encoded_hello = encode('hello') #converts hello into integer
decoded_hello = decode(encoded_hello) 
print(decoded_hello)
The above is a character tokenizer ie it converts each character so we can't use it since it will have a big data

"""

data  = torch.tensor(encode(text),dtype = torch.long)




In [49]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]




In [53]:


x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context= x[:t+1]
    target = y[t]
    print('When input is',context,'target is',target)

When input is tensor([28]) target is tensor(39)
When input is tensor([28, 39]) target is tensor(42)
When input is tensor([28, 39, 42]) target is tensor(39)
When input is tensor([28, 39, 42, 39]) target is tensor(44)
When input is tensor([28, 39, 42, 39, 44]) target is tensor(32)
When input is tensor([28, 39, 42, 39, 44, 32]) target is tensor(49)
When input is tensor([28, 39, 42, 39, 44, 32, 49]) target is tensor(1)
When input is tensor([28, 39, 42, 39, 44, 32, 49,  1]) target is tensor(25)
